In [3]:
from google.colab import drive
#drive.flush_and_unmount()

In [4]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
%cd /content/gdrive/MyDrive/RCGP/RCGP/RCGP-main/

/content/gdrive/MyDrive/RCGP/RCGP/RCGP-main


In [6]:
ls-lrt

total 177
-rw------- 1 root root   482 Aug  2 19:19  README.md
-rw------- 1 root root     0 Aug  2 19:19  __init__.py
drwx------ 2 root root  4096 Aug  2 19:20  rcgp/
drwx------ 2 root root  4096 Aug  2 19:20  experiments/
drwx------ 2 root root  4096 Aug  2 19:48  my_dataset/
-rw------- 1 root root 55269 Aug  5 04:29  Energy_data_focused.xlsx
-rw------- 1 root root 49827 Aug  5 14:50  Energy_data_assymetric.xlsx
-rw------- 1 root root 19615 Aug  5 15:28  Yacht_data_assymetric.xlsx
-rw------- 1 root root 21324 Aug  5 15:32 'Yacht_data_focused (1).xlsx'
-rw------- 1 root root 20772 Aug  5 22:04  Yacht_data_focused.xlsx


In [7]:
import sys
import os

SCRIPT_DIR = os.path.dirname(os.path.abspath(os.getcwd()))
sys.path.append(SCRIPT_DIR)


In [8]:
import tensorflow as tf
import numpy as np
# import gpflow


In [9]:
!pip install check_shapes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 993.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 2.6 MB/s eta 0:00:00


In [10]:
import check_shapes

In [11]:
!pip install gpflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 392.9/392.9 kB 5.5 MB/s eta 0:00:00


In [12]:
from rcgp.rcgp import RCGPR
# from rcgp.rcgp import RCSGPR

from rcgp.w import IMQ
import matplotlib.pyplot as plt

In [13]:
from experiments.uci import datasets as ds
from pathlib import Path

In [14]:
!pip install tueplots
from tueplots import bundles


In [15]:
from time import perf_counter

In [16]:
seed = 10
sigma_n = 0.1

In [17]:
!pwd

/content/gdrive/MyDrive/RCGP/RCGP/RCGP-main


In [18]:
import pandas

In [19]:
import tensorflow as tf
import numpy as np
import pandas as pd
import gpflow


In [20]:
# asymmetric = [ds.boston_asymmetric.create_dataset(Path('/tmp/benchmark_data')),
#                ds.energy_asymmetric.create_dataset(Path('/tmp/benchmark_data')),
#                 ds.synthetic_asymmetric.create_dataset(Path('/tmp/benchmark_data')),
#                   ds.yacht_asymmetric.create_dataset(Path('/tmp/benchmark_data'))]
# time_asymmetric ={}
# mae_asymmetric = {}
# RMSE0 = [[0] * 3 for _ in range(9)]

In [34]:
focused = [ds.boston_focused.create_dataset(Path('/tmp/benchmark_data')),
               ds.energy_focused.create_dataset(Path('/tmp/benchmark_data')),
                ds.synthetic_focused.create_dataset(Path('/tmp/benchmark_data')),
                  ds.yacht_focused.create_dataset(Path('/tmp/benchmark_data'))]
RMSE0 = [[0] * 3 for _ in range(9)]

In [22]:
def RMSE(Eft, yt):
  """
  Calculates the Root Mean Squared Error (RMSE) between two arrays.

  Args:
    Eft: A numpy array of predicted values.
    yt: A numpy array of true values.

  Returns:
    The calculated RMSE.
  """

  rmse = np.sqrt(np.mean((Eft - yt) ** 2))
  return rmse

In [23]:
def MAE(Eft, yt):
  """
  Calculates the Mean Absolute Error (MAE) between two arrays.

  Args:
    Eft: A numpy array of predicted values.
    yt: A numpy array of true values.

  Returns:
    The calculated MAE.
  """

  mae = np.mean(np.abs(Eft - yt))
  return mae

In [24]:
def NLP(means, variances, yTrue):
  """
  Calculates NLP

  Args:
    means: A numpy array of means.
    variances: A numpy array of variances.
    yTrue: A numpy array of true values.

  Returns:
    The calculated value.
  """

  value = 0.5 * np.mean(np.log(2 * np.pi * variances) + ((means - yTrue) ** 2) / variances)
  return value

In [26]:
lengthscale = 1
variance = 1

kernel=gpflow.kernels.SquaredExponential(lengthscales=lengthscale, variance=variance)

In [27]:
from scipy.stats.qmc import Halton
def create_inducing(data):
    n = min(data.N // 2, 200)
    Z = Halton(data.D, scramble=False).random(n)
    lower = np.min(data.X, axis=0)
    upper = np.max(data.X, axis=0)
    Z = Z * (upper - lower) + lower
    return (Z)

In [28]:
def create_rbf(data, rng):
    return gpflow.kernels.SquaredExponential(
        variance=rng.gamma(5.0, 0.2, []),
        lengthscales=rng.gamma(5.0, 0.2, [data.D]),
    )

In [29]:
from rcgp.rcsvgp import RCSGPR
from rcgp.w import IMQ

In [35]:
for dataset in focused:
    print(dataset.name)
    if dataset.name == 'energy_focused':
      for i in range(1):
          print('Iteration',i)
          x = dataset.train.X
          y = dataset.train.Y
          xt = dataset.test.X
          yt = dataset.test.Y

          rng = np.random.default_rng(1235+i)

          inducing_variable = create_inducing(dataset.train)
          variance=rng.gamma(5.0, 0.2, []),
          lengthscales=rng.gamma(5.0, 0.2, [dataset.train.D]),
          noise_variance = 0.01

          # standard_gp = gpflow.models.GPR(
          # (x, y),
          # kernel=gpflow.kernels.SquaredExponential(lengthscales=lengthscales_0,variance = variance_0),
          # noise_variance = 0.01)

          # opt = gpflow.optimizers.Scipy()
          # opt_options = dict()
          # res = opt.minimize(standard_gp.training_loss_closure(), standard_gp.trainable_variables, options=opt_options)


          model = RCSGPR(
          (x, y),
          kernel=create_rbf(dataset.train, rng),
          weighting_function=IMQ(C=np.quantile(np.abs(y), 0.95)),
          noise_variance=sigma_n**2 ,
          inducing_variable = inducing_variable
      )

          gpflow.set_trainable(model.likelihood.variance, False)

          opt = gpflow.optimizers.Scipy()
          log = opt.minimize(model.training_loss_closure(), model.trainable_variables,options={"maxiter": 1_000})
          t_after = perf_counter()

          if isinstance(xt, pd.DataFrame):  # convert xt into tensorflow data types
              xt = xt.values

          Eft, Varft  = model.predict_f(xt)



          # model = RCGPR(
          #     (x, y),
          #     kernel=gpflow.kernels.SquaredExponential(),
          #     # weighting_function=IMQ(1.5),
          #     weighting_function=IMQ(C=np.quantile(np.abs(y), 0.85)),
          #     # weighting_function=IRQ(C=np.quantile(np.abs(Y_train), 0.9)),
          #     noise_variance=standard_gp.likelihood.variance)

          # gpflow.set_trainable(model.likelihood.variance, False)
          # opt = gpflow.optimizers.Scipy()
          # opt_options = dict()
          # res = opt.minimize(model.training_loss_closure(), model.trainable_variables, options=opt_options)
          # print(res)
          # gpflow.utilities.print_summary(model, "notebook")




          Eft, Varft = model.predict_f(xt, full_cov=False)
          std_ft = np.sqrt(Varft)

          RMSE0[i][0]=RMSE(Eft,Varft)
          RMSE0[i][1]=MAE(Eft,Varft)
          RMSE0[i][2]=NLP(Eft,Varft,yt)



boston_focused
energy_focused
Iteration 0
synthetic_focused
yacht_focused


In [36]:
print (RMSE0)



[[0.0038745329641284906, 0.0038745329640772924, 128.3532142155058], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]


In [2]:
from IPython.display import Javascript
display(Javascript('IPython.notebook.execute_cells_above()'))

<IPython.core.display.Javascript object>

In [24]:
df_train = pd.DataFrame({
    'x_0': x[:, 0], 'x_1': x[:, 1], 'x_2': x[:, 2], 'x_3': x[:, 3],
    'x_4': x[:, 4], 'x_5': x[:, 5],
    'y': y.flatten()  # Flatten y to make it 1D
})

df_test = pd.DataFrame({
    'xt_0': xt[:, 0], 'xt_1': xt[:, 1], 'xt_2': xt[:, 2], 'xt_3': xt[:, 3],
    'xt_4': xt[:, 4], 'xt_5': xt[:, 5],
    'yt': yt.flatten()  # Flatten yt to make it 1D
})

# Results = np.zeros((1, 3))
# Results[0,0]=RMSE(Eft, yt)
# Results[0,1]=MAE(Eft, yt)
# Results[0,2]=NLP(Eft,Varft, yt)
# print(Results)

# df_results = pd.DataFrame({
#     'rmse': Results[:, 0], 'mae': Results[:, 1], 'nlp': Results[:, 2]
# })

# Create an ExcelWriter object
with pd.ExcelWriter('Yacht_data_focused.xlsx') as writer:
    df_train.to_excel(writer, sheet_name='Train', index=False)
    df_test.to_excel(writer, sheet_name='Test', index=False)
    # df_results.to_excel(writer, sheet_name='Results', index=False)

In [ ]:
print(RMSE0)
print(i)



[[1.0929273949686509, 0.9487919481334208, 1.1939364336252156], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]
0
